In [1]:
import pandas as pd
import numpy as np

In [2]:
engagement = pd.read_csv('takehome_user_engagement.csv')
engagement = engagement.set_index(pd.DatetimeIndex(engagement['time_stamp']))
users = pd.read_csv('takehome_users.csv',encoding='latin-1')


In [3]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
engagement.head()

,time_stamp,user_id,visited
time_stamp,,,
2014-04-22 03:53:30,2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2013-12-25 03:45:04,2,1


In [5]:
#groups visted by user and then date 
uservisted = engagement.groupby('user_id').resample('7D').sum()


In [6]:
uservisted.head(20)

user_id  visited
user_id time_stamp                  
1       2014-04-22        1        1
2       2013-11-15        2        1
        2013-11-22        0        0
        2013-11-29        2        1
        2013-12-06        2        1
        2013-12-13        0        0
        2013-12-20        2        1
        2013-12-27        2        1
        2014-01-03        2        1
        2014-01-10        0        0
        2014-01-17        0        0
        2014-01-24        0        0
        2014-01-31        2        1
        2014-02-07        6        3
        2014-02-14        2        1
        2014-02-21        0        0
        2014-02-28        0        0
        2014-03-07        4        2
        2014-03-14        0        0
        2014-03-21        0        0

In [7]:
#removes weeks that are null
uservisted = uservisted[uservisted['user_id'].isnull()==False].drop('user_id',axis=1)

In [8]:
#get the adopted users
adopted = uservisted['visited']>=3

In [9]:
adopted.head()

user_id  time_stamp
1        2014-04-22    False
2        2013-11-15    False
         2013-11-22    False
         2013-11-29    False
         2013-12-06    False
Name: visited, dtype: bool

In [10]:
#group them by week 
adopted = adopted.groupby(level=0).apply(np.sum)

In [11]:
#remove users who are not adopted 
adopted = adopted[adopted!=0]
adopted = pd.DataFrame(adopted,index=adopted.index)
adopted.columns = ['Adopteduser']

In [12]:
totaldf = adopted.merge(users,left_index=True,right_on='object_id',how='inner')
totaldf.head()

,Adopteduser,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
1,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
9,53,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0
19,1,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,1.401364e+09,0,0,58,NaN
41,66,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN
42,1,43,2012-12-31 07:13:17,Gomes Beatrice,vgqmpljq@yyyxt.com,GUEST_INVITE,1.366010e+09,0,0,63,149.0


In [13]:
totaldf = adopted.merge(users,left_index=True,right_on='object_id',how='inner')
totaldf['Adopted']=1

In [14]:
users['Adopted']=int(0)
users.iloc[list(totaldf.index),10]=1
finalData = users.set_index('object_id').sort_index()


In [15]:
#removing unesscary features
finalData1 = finalData.drop([ 'name', 'email','last_session_creation_time', 'creation_time'],axis=1)
finalData1.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,Adopted
object_id,,,,,,
1,GUEST_INVITE,1,0,11,10803.0,0
2,ORG_INVITE,0,0,1,316.0,1
3,ORG_INVITE,0,0,94,1525.0,0
4,GUEST_INVITE,0,0,1,5151.0,0
5,GUEST_INVITE,0,0,193,5240.0,0


In [16]:
#fill missing data or and making invited by user id a binaary 
finalData1['invited_by_user_id'] = finalData1['invited_by_user_id'].fillna(0)
finalData1['invited_by_user_id'] = finalData1['invited_by_user_id'].apply(
        lambda x: 1 if x!=0 else 0)

In [17]:
finalData1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 6 columns):
creation_source               12000 non-null object
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            12000 non-null int64
Adopted                       12000 non-null int64
dtypes: int64(5), object(1)
memory usage: 656.2+ KB


In [18]:
#see the values of each column
set(finalData1['creation_source'])

{'GUEST_INVITE',
 'ORG_INVITE',
 'PERSONAL_PROJECTS',
 'SIGNUP',
 'SIGNUP_GOOGLE_AUTH'}

In [19]:
print(set(finalData1['opted_in_to_mailing_list']))
print(set(finalData1['invited_by_user_id']))
print(set(finalData1['Adopted']))
print(set(finalData1['org_id']))

{0, 1}
{0, 1}
{0, 1}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217

In [20]:
activeuser = finalData1[finalData1['Adopted']==1]

In [21]:
# is seems like adopted users a rare feature
len(activeuser)

1472

In [22]:
org = finalData1.groupby('org_id').sum()
print(org)

        opted_in_to_mailing_list  enabled_for_marketing_drip  \
org_id                                                         
0                             77                          42   
1                             45                          20   
2                             36                          27   
3                             41                          16   
4                             45                          25   
5                             34                          21   
6                             33                          17   
7                             29                          23   
8                             21                          10   
9                             36                          15   
10                            27                          19   
11                            18                           9   
12                            20                          13   
13                            22        

In [23]:
#organize/catorgirze the organizaton feature
org_cata = []
for i in finalData1['org_id']:
    cata = org.loc[i,'Adopted']
    org_cata.append(cata)
finalData1['org_id'] = org_cata
finalData1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 6 columns):
creation_source               12000 non-null object
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            12000 non-null int64
Adopted                       12000 non-null int64
dtypes: int64(5), object(1)
memory usage: 656.2+ KB


In [24]:
#get the dummies features
finalData2 = pd.get_dummies(data=finalData1, columns=['creation_source'])

In [25]:
finalData2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 10 columns):
opted_in_to_mailing_list              12000 non-null int64
enabled_for_marketing_drip            12000 non-null int64
org_id                                12000 non-null int64
invited_by_user_id                    12000 non-null int64
Adopted                               12000 non-null int64
creation_source_GUEST_INVITE          12000 non-null uint8
creation_source_ORG_INVITE            12000 non-null uint8
creation_source_PERSONAL_PROJECTS     12000 non-null uint8
creation_source_SIGNUP                12000 non-null uint8
creation_source_SIGNUP_GOOGLE_AUTH    12000 non-null uint8
dtypes: int64(5), uint8(5)
memory usage: 621.1 KB


In [26]:
finalData2 = pd.get_dummies(data=finalData2, columns=['org_id'])

In [27]:
#the set used for modeling/Data Modeling 
finalData2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 25 columns):
opted_in_to_mailing_list              12000 non-null int64
enabled_for_marketing_drip            12000 non-null int64
invited_by_user_id                    12000 non-null int64
Adopted                               12000 non-null int64
creation_source_GUEST_INVITE          12000 non-null uint8
creation_source_ORG_INVITE            12000 non-null uint8
creation_source_PERSONAL_PROJECTS     12000 non-null uint8
creation_source_SIGNUP                12000 non-null uint8
creation_source_SIGNUP_GOOGLE_AUTH    12000 non-null uint8
org_id_0                              12000 non-null uint8
org_id_1                              12000 non-null uint8
org_id_2                              12000 non-null uint8
org_id_3                              12000 non-null uint8
org_id_4                              12000 non-null uint8
org_id_5                              12000 non-null uint8
org_id

In [28]:
finalData2.head()

,opted_in_to_mailing_list,enabled_for_marketing_drip,invited_by_user_id,Adopted,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,org_id_0,...,org_id_6,org_id_7,org_id_8,org_id_9,org_id_10,org_id_11,org_id_12,org_id_13,org_id_14,org_id_16
object_id,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
#try random forrest first 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [30]:
y = finalData2.pop('Adopted')
x = finalData2

In [31]:
#train test split on the features/labels 
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.30,random_state=42)

In [32]:
clf = RandomForestClassifier(class_weight='balanced')

In [33]:
clf.fit(x_train,y_train)

C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [34]:
ypred = clf.predict(x_test)

In [35]:
print(classification_report(y_test, ypred)) 

              precision    recall  f1-score   support

           0       0.90      0.63      0.74      3151
           1       0.16      0.48      0.24       449

   micro avg       0.61      0.61      0.61      3600
   macro avg       0.53      0.56      0.49      3600
weighted avg       0.80      0.61      0.68      3600



In [36]:
#hyper parameter tuning for random forest 
n_estimators = [4, 8, 32, 64, 100, 200, 300]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
grid1 = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [37]:
gridbest = GridSearchCV(estimator = clf, scoring='accuracy', param_grid = grid1,  n_jobs = -1, verbose = 2)

In [38]:
gridbest.fit(x_train,y_train)

C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 3024 candidates, totalling 9072 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   39.4s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 5816 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 6829 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 7922 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done 9072 out of 9072 | elapsed: 18.0min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [4, 8, 32, 64, 100, 200, 300], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=2)

In [39]:
gridbest.best_params_

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 4}

In [40]:
gridbest.best_score_

0.6371428571428571

In [43]:
#try random forest with best parameters
clf1 = RandomForestClassifier(class_weight='balanced', max_depth=20, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators=4, bootstrap=True)

In [44]:
clf1.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=20, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=4, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [45]:
ypred = clf1.predict(x_test)

In [46]:
#does not seem like performing model
print(classification_report(y_test, ypred)) 

              precision    recall  f1-score   support

           0       0.90      0.66      0.76      3151
           1       0.16      0.46      0.24       449

   micro avg       0.64      0.64      0.64      3600
   macro avg       0.53      0.56      0.50      3600
weighted avg       0.80      0.64      0.70      3600



In [47]:
#try gradient boosting model as it predict on the error/term
from sklearn.ensemble import GradientBoostingClassifier

In [53]:
gbm = GradientBoostingClassifier(max_features='sqrt')

In [54]:
gbm.fit(x_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [55]:
ypred = gbm.predict(x_test)

In [56]:
#seems the same as rf
print(classification_report(y_test, ypred)) 

              precision    recall  f1-score   support

           0       0.88      1.00      0.93      3151
           1       0.00      0.00      0.00       449

   micro avg       0.88      0.88      0.88      3600
   macro avg       0.44      0.50      0.47      3600
weighted avg       0.77      0.88      0.82      3600



C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [68]:
#hyperparameter tune the gb model
gvparams = {'n_estimators':[40,45,50,55,60],'learning_rate':[.01,0.02,0.03,0.04,0.05],
              'max_depth':[1,2,3,4,5]}

In [58]:
bestgrid1 = GridSearchCV(estimator=gbm, param_grid=gvparams, scoring='accuracy')

In [59]:
bestgrid1.fit(x_train,y_train)

C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sam...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [40, 45, 50, 55, 60], 'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05], 'max_depth': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [60]:
bestgrid1.best_params_

{'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 40}

In [61]:
#seems like a better performer
bestgrid1.best_score_

0.8782142857142857

In [62]:
#use the best parameters
gbm1 = GradientBoostingClassifier(max_features='sqrt', learning_rate=0.01, max_depth=1, n_estimators=40)

In [63]:
gbm1.fit(x_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=1,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=40,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [65]:
ypred = gbm1.predict(x_test)

In [66]:
#same classfication as above
print(classification_report(y_test, ypred)) 

              precision    recall  f1-score   support

           0       0.88      1.00      0.93      3151
           1       0.00      0.00      0.00       449

   micro avg       0.88      0.88      0.88      3600
   macro avg       0.44      0.50      0.47      3600
weighted avg       0.77      0.88      0.82      3600



C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [67]:
ordercoef = pd.DataFrame(gbm1.feature_importances_).transpose()
ordercoef.columns = list(finalData2.columns)
ordercoef.index = ['GBM'] 
ordercoef.transpose().sort_values(by='GBM',ascending=False)

,GBM
creation_source_PERSONAL_PROJECTS,0.313022
org_id_0,0.179382
org_id_5,0.153601
org_id_6,0.124505
org_id_1,0.090779
creation_source_SIGNUP_GOOGLE_AUTH,0.047849
org_id_11,0.025215
org_id_14,0.014154
org_id_7,0.012982
org_id_2,0.012038


It seems like that creation_source_PERSONAL_PROJECTS is the biggest factor in making a user an adopted user followed by them being in org 0 and org 5. So if you attract more user who use this service for personal projects then there will be more adopted users. In addition, if work with org 0 and org 5 to get more users then the company can gain more adopted users. Some improvements that could be done on this assignement is to try tpot autoML to get the best performing model or try more hyperparameter tuning. 